In [1]:
import math
import numpy as np
np.random.seed(23456)
from matplotlib import pyplot as plt
from source import *

In [2]:
S0=100
sigma = 0.20978
r=0.01
div=0.001
mu = r - div
T=1
K=110

I=1000000
M=12
dt=T/M
df=math.exp(-r*dt)

In [3]:
## Closed form
def cnorm(x):
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0

d1 = (np.log(S0/K) + (r+((sigma**2)/2))*T)/(sigma* np.sqrt(T))
d2 = (np.log(S0/K) + (r-((sigma**2)/2))*T)/(sigma* np.sqrt(T))

p = S0*cnorm(d1) - K*np.exp(-r*T)*cnorm(d2)

print(p)


4.981503039726142


In [4]:
## MC - Only simulate last timepoint
N = np.random.standard_normal(I)
Sum = 0
for i in N:
    S = S0*np.exp((r - sigma*sigma/2)*T + sigma*np.sqrt(T)*i)
    Payoff = max(S-K,0)
    Sum = Sum + Payoff

Sum = Sum*np.exp(-r*T)
print(Sum/len(N))

4.985534942029281


In [5]:
## MC - Simulate all timepoints
delta_X = (r - sigma*sigma/2)*dt + (sigma * np.sqrt(dt)* np.random.standard_normal((I,M)))
delta_X = np.hstack((np.reshape(np.zeros(I),(-1,1)), delta_X))
S=S0*np.exp(np.cumsum(delta_X, axis=1))
price = np.exp(-r*T)*sum(np.maximum(S.T[-1]-K,0))/len(S.T[-1])
print(price)

4.979386036259684


In [21]:
## American Option
delta_X = (r - sigma*sigma/2)*dt + (sigma * np.sqrt(dt)* np.random.standard_normal((I,M)))
delta_X = np.hstack((np.reshape(np.zeros(I),(-1,1)), delta_X))
S=S0*np.exp(np.cumsum(delta_X, axis=1))

h = np.maximum(K-S,0)
V= h.T[-1]
for i in range(M-1,0,-1):
    rg = np.polyfit(S.T[i],V*df,5)
    C = np.polyval(rg,S.T[i])
    V=np.where(h.T[i]>C,h.T[i],V*df)

V0 = df * np.average(V)
print(V0)

13.978288912276618


In [25]:
## American Option using function
trd = AmericanOption(T,K,PayoffType.Put)
lsmcAmerican(S0, r, div, sigma, I, M, trd)


13.982305124646476